In [13]:
import requests
from io import BytesIO
import pandas as pd
import re
import os
from zipfile import ZipFile
from datetime import datetime

USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:69.0) Gecko/20100101 firefox/69.0"
PATH = "./"

API_KEY = "5b2af3ba6b092b9916face3fe4ff49150bc7278d"

corp_code = "005930"
page_count = "10"
reports = ["연결 재무상태표", "연결 손익계산서", "연결 포괄손익계산서"]
bgn_de = "19990101"
end_de = datetime.today().strftime("%Y%m%d")

# 모든 기업의 코드

In [14]:
url = "https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={}".format(API_KEY)

resp = requests.get(url, headers={"user-agent": User_agent})

webpage = resp.content

zf = ZipFile(BytesIO(webpage), "r")
fileinfo = zf.infolist()[0]
corpCode = zf.read(fileinfo).decode('UTF-8')

# 기업 코드 (주의. 주식 코드와 다름)
corp_code_list = re.findall(r'<corp_code>(.*?)</corp_code>', corpCode)
# # 정식명칭
# corp_name_list = re.findall(r'<corp_name>(.*?)</corp_name>', corpCode)

# corp_dict = dict(zip(corp_name_list, corp_code_list))

## 주식 시장에 따라 나누기

In [15]:
def download_company_info(corp_code, api_key=API_KEY, user_agent=USER_AGENT):
    url = "https://opendart.fss.or.kr/api/company.json?crtfc_key={}&corp_code={}".format(api_key, corp_code)

    resp = requests.get(url, headers={"user-agent": User_agent})

    webpage = resp.content.decode('UTF-8')

    # 종목명(상장사) 또는 약식명칭(기타법인)
    stock_name = re.findall(r'"stock_name":"(.*?)",', webpage)
    # 상장회사인 경우 주식의 종목코드
    stock_code = re.findall(r'"stock_code":"(.*?)",', webpage)
    # 법인구분 : Y(유가), K(코스닥), N(코넥스), E(기타)
    corp_cls = re.findall(r'"corp_cls":"(.*?)",', webpage)
    return stock_name, stock_code, corp_cls

In [16]:
# 유가증권시장 상장 기업
company_Y_dict = {}
# 코스닥 상장 기업
company_K_dict = {}
# 코넥스 상장 기업
company_N_dict = {}
# 기타 기업(비상장)
company_E_dict = {}

for i, corp_code in enumerate(corp_code_list):
    print(">", end="")
    stock_name, stock_code, corp_cls = download_company_info(corp_code)
    if corp_cls == 'Y':
        company_Y_dict[corp_code] = [stock_name, stock_code, corp_cls]
    elif corp_cls == 'K':
        company_K_dict[corp_code] = [stock_name, stock_code, corp_cls]
    elif corp_cls == 'N':
        company_N_dict[corp_code] = [stock_name, stock_code, corp_cls]
    elif corp_cls == 'E':
        company_E_dict[corp_code] = [stock_name, stock_code, corp_cls]
    else:
        continue


KeyboardInterrupt: 

In [ ]:
company_Y_dict

# 재무제표

In [163]:
def download_report(corp_name, period, rcp_no, dcm_no, sheets, user_agent=USER_AGENT):
    url = "https://dart.fss.or.kr/pdf/download/excel.do?rcp_no={}&dcm_no={}&lang=ko".format(rcp_no, dcm_no)

    resp = requests.get(url, headers={"user-agent": user_agent})
    table = BytesIO(resp.content)

    path = "".join([PATH, corp_name, "/"])
    if not os.path.isdir(path):
        os.mkdir(path)
    
    for sheet in sheets:
        data = pd.read_excel(table, sheet_name=sheet, skiprows=5)
        data.to_csv(path+"_".join([corp_name, period[-6:-4]+period[-3:-1], sheet])+".csv", encoding="euc-kr")

In [169]:
def download_report_of_company(corp_name, corp_code, bgn_de, end_de, page_count, reports, api_key=API_KEY):
    url = "https://opendart.fss.or.kr/api/list.xml?crtfc_key={}&corp_code={}&bgn_de={}&end_de={}&pblntf_detail_ty=A001&pblntf_detail_ty=A002&pblntf_detail_ty=A003&page_count={}".format(api_key, corp_code, bgn_de, end_de, page_count)

    resp = requests.get(url)
    webpage = resp.content.decode('UTF-8')

    # rcp_no: 제출 일차
    rcept_no_list = re.findall(r'<rcept_no>(.*?)</rcept_no>', webpage)
    if len(rcept_no_list) == 0:
        return False
    # period: 시기
    period_list = re.findall(r'<report_nm>(.*?)</report_nm>', webpage)
    
    # dcm_no 문서 번호
    dcm_no_list = []
    for rcpNo in rcept_no_list:
        resp = requests.get("https://dart.fss.or.kr/dsaf001/main.do?rcpNo={}".format(rcpNo))
        webpage = resp.content.decode('UTF-8')
        dcm_no = re.findall(r"'dcmNo'] = \"(.*?)\";", webpage)[0]
        dcm_no_list.append(dcm_no)

    report_dict = dict(zip(period_list, zip(rcept_no_list, dcm_no_list)))

    for key in report_dict.keys():
        download_report(corp_name, key, report_dict[key][0], report_dict[key][1], reports)
    return True

In [170]:
"""
df = pd.read_csv('pocket.txt')

for period, rcp_no, dcm_no in zip(df["period"].values, df["rcp_no"].values, df["dcm_no"].values):
    download_excel("삼성전자", str(period), str(rcp_no), str(dcm_no))
"""



'\ndf = pd.read_csv(\'pocket.txt\')\n\nfor period, rcp_no, dcm_no in zip(df["period"].values, df["rcp_no"].values, df["dcm_no"].values):\n    download_excel("삼성전자", str(period), str(rcp_no), str(dcm_no))\n'

# 전체 기업 제무제표 가져오기

In [ ]:
for corp_name in corp_dict:
    print(corp_name, end=" ")
    isReport = download_report_of_company(corp_name, corp_dict[corp_name], bgn_de, end_de, page_count, reports)
    if isReport == False:
        del corp_dict[corp_name]